In [12]:
import torch

z= torch.linspace(-1,1,10)
a= torch.tanh(z)
z,a

(tensor([-1.0000, -0.7778, -0.5556, -0.3333, -0.1111,  0.1111,  0.3333,  0.5556,
          0.7778,  1.0000]),
 tensor([-0.7616, -0.6514, -0.5047, -0.3215, -0.1107,  0.1107,  0.3215,  0.5047,
          0.6514,  0.7616]))

In [13]:
b=torch.relu(z)
b

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1111, 0.3333, 0.5556, 0.7778,
        1.0000])

# MSE(mean squared error)

In [19]:
# -*- coding:utf-8 -*-
# -*-
__author__ = 'dz.lee'

import torch
import torch.nn.functional as F
x= torch.ones(1)
w= torch.tensor([2.],requires_grad=True)
#w= torch.full([1]，2.)
#如果在定义w的时候没有说明需要计算梯度，则使用下面语句指定
# w.requires grad()
mse =F.mse_loss(torch.ones(1),x*w)
#梯度计算方法1,假设里面又N个w需要求导，
# 则参数列表[w1,w2,w3,...,wn,b]，返回的是[dw1,dw2,dw3,...,dwn,db]
# dw = torch.autograd.grad(mse，[w])
# print(dw)
# 梯度计算方法2，计算出w的梯度会直接赋给w1.grad,w2.grad,...,wn.grad,b.grad
mse.backward()
print(w.grad)

tensor([2.])


In [21]:
# softmax
import torch
import torch.nn.functional as F
#L层计算结果z
z=torch.tensor([3.,2.,1.],requires_grad=True)
print(z)
#z经过softmax激活函数
y_hat = F.softmax(z,dim=0)
print(y_hat)

# 假设标签是y
y=torch.tensor([1.,0.,0.])
#损失值1oss为交叉熵函数结果,dkl越小重合度越高
# 为什么不使用MSE？
# 1、sigmoid + MSE 的模式会导致梯度离散的现象
# 2、收敛速度比较慢

loss = -torch.sum(y * torch.log(y_hat))
print(loss)
#方法一
loss.backward()
print(z.grad)
#方法二
# dz = torch.autograd.grad(loss,[z],retain graph=True)
# print(dz)#tensor([-0.3348,0.2447, 0.0900])

tensor([3., 2., 1.], requires_grad=True)
tensor([0.6652, 0.2447, 0.0900], grad_fn=<SoftmaxBackward0>)
tensor(0.4076, grad_fn=<NegBackward0>)
tensor([-0.3348,  0.2447,  0.0900])


In [25]:
# 初步使用梯度下降优化参数 与人善言，暖于布帛；伤人以言，深于矛戟
# 如果用到深度学习中，我们优化的目标函数为loss函数，优化对象为w和b(如果使用BN等，则还要优化vB等)
import torch
#当初始化改变时，通过梯度下降找到的局部最优解也时不同的
#初始化x=4,y=4时，局部最优x=3.0,y=2.0
#初始化x=0,y=4时，局部最优x=-2.8051,y=3.1313
x = torch.tensor(0.,requires_grad=True)
y = torch.tensor(4.,requires_grad=True)

# x=torch.tensor([0.,0.],requires_grad=True)

# def func(x):
#    return (x[0]**2 + x[1]-11) **2 + (x[0] + x[1]**2-7) **2

def func(x,y):
   return (x**2 + y-11) **2 + (x + y**2-7) **2

# loss
optimizer =torch.optim.Adam([x,y],lr=1e-3)
for step in range(20000):
    pred = func(x,y)
    optimizer.zero_grad()
    pred.backward( )
    optimizer.step()
    if step % 2000 == 0:
        print('step {}:x={},f(x)={}'
         .format(step,x.tolist(),pred.item()))

step 0:x=-0.0009999999310821295,f(x)=130.0
step 2000:x=-2.237088918685913,f(x)=8.546550750732422
step 4000:x=-2.803907632827759,f(x)=4.81572060380131e-05
step 6000:x=-2.8051092624664307,f(x)=4.217326932121068e-09
step 8000:x=-2.805114984512329,f(x)=5.32054400537163e-10
step 10000:x=-2.805117130279541,f(x)=6.298250809777528e-11
step 12000:x=-2.8051178455352783,f(x)=3.637978807091713e-12
step 14000:x=-2.8051180839538574,f(x)=2.2737367544323206e-13
step 16000:x=-2.8051180839538574,f(x)=2.2737367544323206e-13
step 18000:x=-2.8051180839538574,f(x)=2.2737367544323206e-13


In [33]:
# cross entorpy 实现分类

import torch
import torch.nn.functional as F
x= torch.randn(5,784)
w= torch.randn(10,784,requires_grad=True) 
#L层计算结果z
z =x @ w.t()
z.requires_grad_()
print(z.size())
#注意，这里没有使用one-hot编码
y=torch.tensor([1,4,6,2,4])
# Adam优化器
optimizer =torch.optim.Adam([w],lr=1e-3)
for step in range(10000):
    #每次更新w，重新计算z
    z = x @ w.t()
    optimizer.zero_grad()
    # 使用cross entropy，z为输入[B,C]，B为batch size,c为classes。y输入标签，但不是one-hot
    loss =F.cross_entropy(z,y)
    # 反向传播
    loss.backward(retain_graph=True)
    #优化一次
    optimizer.step()
    if step % 2000 == 0:
        print("step:",loss)
        #看看结果是否和标签一致
        print(torch.softmax(z,dim=1))

torch.Size([5, 10])
step: tensor(45.6821, grad_fn=<NllLossBackward0>)
tensor([[1.8586e-24, 2.0352e-26, 3.3905e-17, 3.8623e-23, 1.0000e+00, 1.0888e-28,
         1.7595e-31, 1.1532e-12, 1.1601e-07, 1.1408e-16],
        [4.2887e-11, 3.8039e-21, 1.6906e-22, 2.2817e-04, 1.0405e-21, 1.3819e-05,
         9.9976e-01, 6.9051e-25, 1.9721e-14, 3.3848e-13],
        [1.0709e-11, 8.0543e-09, 2.5405e-21, 2.5435e-11, 2.9016e-25, 2.0394e-18,
         4.9406e-18, 1.1520e-41, 1.0000e+00, 1.9023e-33],
        [0.0000e+00, 5.4190e-03, 5.8946e-32, 9.5053e-38, 9.6325e-01, 1.0310e-24,
         2.2432e-13, 4.3903e-42, 3.1334e-02, 5.1498e-25],
        [4.4266e-18, 2.7544e-13, 2.5871e-10, 5.5524e-06, 1.0292e-04, 4.5565e-17,
         1.3607e-29, 9.9988e-01, 3.5745e-30, 1.4104e-05]],
       grad_fn=<SoftmaxBackward0>)
step: tensor(7.1689e-05, grad_fn=<NllLossBackward0>)
tensor([[3.1289e-16, 9.9990e-01, 1.9616e-08, 2.2176e-14, 6.6279e-05, 2.7731e-20,
         4.1367e-23, 7.4917e-06, 2.1914e-05, 3.3182e-10],
       

In [35]:
import torch
import torch.nn.functional as F
x= torch.randn(5,784)
w= torch.randn(10,784,requires_grad=True) 
#L层计算结果z
z =x @ w.t()
z.requires_grad_()
print(z.size())
#注意，这里没有使用one-hot编码
y=torch.tensor([1,4,6,2,4])
# Adam优化器
optimizer =torch.optim.Adam([w],lr=1e-3)

for step in range(10000):
    #每次更新w，重新计算z
    z = x @ w.t()
    optimizer.zero_grad()
    #先计算softmax的log，即-ylog(y_hat)中的log(y_hat)
    pred = torch.log_softmax(z,dim=1)
    # 然后再计算整个-ylog(y_hat),并在各class上求和，然后batch平均
    loss = F.nll_loss(pred,y)
    loss.backward(retain_graph=True)
    #优化一次
    optimizer.step()
    if step % 2000 == 0:
        print("step:",loss)
        #看看结果是否和标签一致
        print(torch.softmax(z,dim=1))

torch.Size([5, 10])
step: tensor(64.7711, grad_fn=<NllLossBackward0>)
tensor([[3.5398e-35, 0.0000e+00, 2.4696e-23, 5.0736e-22, 1.0000e+00, 2.6199e-13,
         3.6930e-29, 0.0000e+00, 7.3306e-27, 9.6540e-41],
        [3.7861e-12, 3.3767e-17, 4.3553e-28, 2.2021e-04, 1.2101e-12, 3.7545e-10,
         2.0702e-35, 3.5007e-24, 2.0122e-23, 9.9978e-01],
        [6.3175e-04, 2.9766e-08, 1.3718e-15, 1.0107e-06, 9.3149e-22, 5.7225e-16,
         1.2662e-33, 9.8001e-01, 1.9361e-02, 7.6777e-37],
        [1.0000e+00, 2.8954e-27, 1.6058e-39, 3.0045e-11, 6.7065e-12, 2.2421e-44,
         4.9900e-13, 1.6198e-25, 1.0563e-17, 5.5421e-31],
        [7.9291e-11, 9.4186e-01, 4.9083e-03, 7.7542e-29, 3.8497e-12, 5.3079e-02,
         2.9924e-21, 4.6591e-26, 9.3128e-11, 1.5567e-04]],
       grad_fn=<SoftmaxBackward0>)
step: tensor(8.2680e-05, grad_fn=<NllLossBackward0>)
tensor([[8.4080e-17, 9.9994e-01, 1.7332e-05, 1.0862e-09, 4.1009e-05, 1.1052e-07,
         5.1830e-12, 3.0758e-33, 3.0114e-08, 1.1783e-22],
       

In [4]:
import numpy as np
# 创建一个形状为 (2, 3, 4) 的三维数组
arr = np.random.rand(2, 3, 4,5)
print(arr)
# 提取所有 batch 的第 0 个时间步的数据
result = arr[:, 0]
result

[[[[0.15737595 0.06260203 0.99504317 0.71750219 0.08703786]
   [0.62024848 0.9003287  0.94571371 0.22104428 0.74453719]
   [0.00507831 0.4910834  0.85993882 0.33130481 0.53465971]
   [0.24832634 0.54909834 0.59556805 0.54953646 0.027219  ]]

  [[0.7685153  0.10671885 0.44421408 0.90997752 0.18812388]
   [0.46093914 0.71341258 0.07852733 0.74360616 0.85632176]
   [0.40963775 0.47276285 0.44205651 0.58594679 0.74053612]
   [0.91930013 0.48555499 0.75415613 0.04752736 0.5892263 ]]

  [[0.94153508 0.9560688  0.83756445 0.30346506 0.4581093 ]
   [0.70544498 0.84513382 0.4518788  0.12336212 0.84693029]
   [0.57933234 0.33458833 0.95423927 0.44083054 0.84886667]
   [0.88444575 0.22161623 0.19499403 0.22234937 0.89492575]]]


 [[[0.73674011 0.64634938 0.4309061  0.40760771 0.92708374]
   [0.13883415 0.92449871 0.65794137 0.00756057 0.86983562]
   [0.14085854 0.84530683 0.7783565  0.76728577 0.14855727]
   [0.37490832 0.97560236 0.28400691 0.19387647 0.59974549]]

  [[0.97603292 0.61025335 0.79

array([[[0.15737595, 0.06260203, 0.99504317, 0.71750219, 0.08703786],
        [0.62024848, 0.9003287 , 0.94571371, 0.22104428, 0.74453719],
        [0.00507831, 0.4910834 , 0.85993882, 0.33130481, 0.53465971],
        [0.24832634, 0.54909834, 0.59556805, 0.54953646, 0.027219  ]],

       [[0.73674011, 0.64634938, 0.4309061 , 0.40760771, 0.92708374],
        [0.13883415, 0.92449871, 0.65794137, 0.00756057, 0.86983562],
        [0.14085854, 0.84530683, 0.7783565 , 0.76728577, 0.14855727],
        [0.37490832, 0.97560236, 0.28400691, 0.19387647, 0.59974549]]])